DataLoader for Prescription Word Dataset

In [ ]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image, ImageOps

# Resize and pad images to (128, 32) - Same as old CRNN input size
def resize_and_pad(img, size=(128, 32)):
    img = img.convert("L")  # Convert to grayscale
    w, h = img.size
    scale = size[1] / h
    new_w = int(w * scale)
    img = img.resize((new_w, size[1]), Image.BILINEAR)

    if new_w > size[0]:
        img = img.resize(size, Image.BILINEAR)
    else:
        delta_w = size[0] - new_w
        padding = (delta_w // 2, 0, delta_w - delta_w // 2, 0)
        img = ImageOps.expand(img, padding, fill=255)

    return img

# Custom Dataset
class PrescriptionWordsDataset(Dataset):
    def __init__(self, csv_path, img_folder, img_size=(128, 32)):
        self.data = pd.read_csv(csv_path)
        self.img_folder = img_folder
        self.img_size = img_size
        self.transform = transforms.Compose([
            transforms.Lambda(lambda img: resize_and_pad(img, size=self.img_size)),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name, label = self.data.iloc[idx]
        img_path = os.path.join(self.img_folder, img_name)

        image = Image.open(img_path).convert("RGB")
        image = self.transform(image)

        return image, label


Label Encoder for Characters

In [ ]:
# Label Encoder
class LabelEncoder:
    def __init__(self):
        self.characters = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789-.,:/()"
        self.char2idx = {char: idx + 1 for idx, char in enumerate(self.characters)}  # 0 reserved for blank
        self.idx2char = {idx + 1: char for idx, char in enumerate(self.characters)}
        self.blank_label = 0

    def encode(self, text):
        return [self.char2idx[char] for char in text if char in self.char2idx]

    def decode(self, indices):
        return ''.join([self.idx2char.get(idx, '') for idx in indices if idx != self.blank_label])

    def get_vocab_size(self):
        return len(self.char2idx) + 1


Build CRNN Model

In [ ]:
import torch.nn as nn

class CRNN(nn.Module):
    def __init__(self, img_height, num_classes):
        super(CRNN, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 64, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(64, 128, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(128, 256, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d((2, 1)),

            nn.Conv2d(256, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d((2, 1)),

            nn.Conv2d(512, 512, 2, 1, 0),
            nn.ReLU()
        )

        self.rnn1 = nn.LSTM(512, 256, bidirectional=True, batch_first=True)
        self.rnn2 = nn.LSTM(512, 256, bidirectional=True, batch_first=True)



        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
      x = self.cnn(x)
      b, c, h, w = x.size()
      assert h == 1, "Height must be 1 after convs"
      x = x.squeeze(2)  # Remove height dimension
      x = x.permute(0, 2, 1)  # [batch, width, channels]

      x, _ = self.rnn1(x)
      x, _ = self.rnn2(x)
      x = self.fc(x)
      return x


In [ ]:
import torch.optim as optim
from torch.nn import CTCLoss

# Paths
csv_path = "/content/drive/MyDrive/Individual_Research_Project/prescription_words_dataset.csv"
img_folder = "/content/drive/MyDrive/Individual_Research_Project/cropped_prescription_words"

# Parameters
batch_size = 32
epochs = 20
learning_rate = 0.001
img_height = 32

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Load dataset
train_dataset = PrescriptionWordsDataset(csv_path, img_folder)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Label encoder
encoder = LabelEncoder()

# Model
vocab_size = encoder.get_vocab_size()
model = CRNN(img_height=img_height, num_classes=vocab_size).to(device)

# Loss and Optimizer
criterion = CTCLoss(blank=0, zero_infinity=True)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


Using device: cuda


Model Training

In [ ]:
# Training Loop
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for images, labels in train_loader:
        # Encode labels
        targets = [encoder.encode(label) for label in labels]
        target_lengths = torch.IntTensor([len(t) for t in targets])
        targets = torch.IntTensor([i for seq in targets for i in seq])

        images = images.to(device)
        targets = targets.to(device)
        target_lengths = target_lengths.to(device)

        # Forward
        outputs = model(images)
        input_lengths = torch.full(size=(outputs.size(0),), fill_value=outputs.size(1), dtype=torch.long).to(device)

        loss = criterion(outputs.log_softmax(2).transpose(0, 1), targets, input_lengths, target_lengths)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs} | Loss: {avg_loss:.4f}")


Epoch 1/20 | Loss: 3.9680
Epoch 2/20 | Loss: 2.9875
Epoch 3/20 | Loss: 2.7479
Epoch 4/20 | Loss: 2.5794
Epoch 5/20 | Loss: 2.4441
Epoch 6/20 | Loss: 2.3538
Epoch 7/20 | Loss: 2.2453
Epoch 8/20 | Loss: 2.1529
Epoch 9/20 | Loss: 2.0479
Epoch 10/20 | Loss: 1.9228
Epoch 11/20 | Loss: 1.8115
Epoch 12/20 | Loss: 1.7096
Epoch 13/20 | Loss: 1.5843
Epoch 14/20 | Loss: 1.5002
Epoch 15/20 | Loss: 1.3817
Epoch 16/20 | Loss: 1.3185
Epoch 17/20 | Loss: 1.2008
Epoch 18/20 | Loss: 1.1320
Epoch 19/20 | Loss: 1.0544
Epoch 20/20 | Loss: 0.9860


In [ ]:
os.makedirs("/content/drive/MyDrive/Individual_Research_Project/models/", exist_ok=True)
torch.save(model.state_dict(), "/content/drive/MyDrive/Individual_Research_Project/models/crnn_prescription_final.pth")
print("Model saved successfully!")


Model saved successfully!
